In [117]:
import numpy as np
import pandas as pd
# for data cleaning 
import re
from gensim.parsing.preprocessing import remove_stopwords # using gensim for stopwords removal 

In [118]:
anime = pd.read_csv('animes.csv')

In [119]:
anime.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']",6-Jan-17,1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


In [120]:
anime.shape

(19311, 12)

In [121]:
anime.head(1)

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...


In [122]:
# Cols that are needed for reccomendation system 
# Genre
# id
# synopsis 
# title 

anime = anime[['uid', 'title', 'synopsis', 'genre']]

In [123]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19311 entries, 0 to 19310
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   uid       19311 non-null  int64 
 1   title     19311 non-null  object
 2   synopsis  18336 non-null  object
 3   genre     19311 non-null  object
dtypes: int64(1), object(3)
memory usage: 603.6+ KB


In [124]:
# This is the dataframe where we will build our reccomendation system

anime.head()

,uid,title,synopsis,genre
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun..."
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun..."
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F..."
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ..."
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']"


In [125]:
# Checking for missing data 
anime.isnull().sum()

uid           0
title         0
synopsis    975
genre         0
dtype: int64

In [126]:
anime.dropna(inplace=True) # dropping rows w/ missing vals 

In [127]:
# Checking for duplicate data
anime.duplicated().sum()

2883

In [128]:
anime = anime.drop_duplicates(keep='first') # keep first duplicate row

In [129]:
anime.iloc[0].genre # The way genre is formatted is ok, we don't have to change anything

"['Comedy', 'Sports', 'Drama', 'School', 'Shounen']"

In [130]:
anime['synopsis'][0]

"Following their participation at the Inter-High, the Karasuno High School volleyball team attempts to refocus their efforts, aiming to conquer the Spring tournament instead.  \r\n \r\nWhen they receive an invitation from long-standing rival Nekoma High, Karasuno agrees to take part in a large training camp alongside many notable volleyball teams in Tokyo and even some national level players. By playing with some of the toughest teams in Japan, they hope not only to sharpen their skills, but also come up with new attacks that would strengthen them. Moreover, Hinata and Kageyama attempt to devise a more powerful weapon, one that could possibly break the sturdiest of blocks.  \r\n \r\nFacing what may be their last chance at victory before the senior players graduate, the members of Karasuno's volleyball team must learn to settle their differences and train harder than ever if they hope to overcome formidable opponents old and new—including their archrival Aoba Jousai and its world-class 

In [131]:
anime['synopsis'] = anime['synopsis'].apply(lambda x: x.split()) # Turning each word into a list 

In [132]:
anime.head()

,uid,title,synopsis,genre
0,28891,Haikyuu!! Second Season,"[Following, their, participation, at, the, Int...","['Comedy', 'Sports', 'Drama', 'School', 'Shoun..."
1,23273,Shigatsu wa Kimi no Uso,"[Music, accompanies, the, path, of, the, human...","['Drama', 'Music', 'Romance', 'School', 'Shoun..."
2,34599,Made in Abyss,"[The, Abyss—a, gaping, chasm, stretching, down...","['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F..."
3,5114,Fullmetal Alchemist: Brotherhood,"[""In, order, for, something, to, be, obtained,...","['Action', 'Military', 'Adventure', 'Comedy', ..."
4,31758,Kizumonogatari III: Reiketsu-hen,"[After, helping, revive, the, legendary, vampi...","['Action', 'Mystery', 'Supernatural', 'Vampire']"


In [133]:
anime.head()

,uid,title,synopsis,genre
0,28891,Haikyuu!! Second Season,"[Following, their, participation, at, the, Int...","['Comedy', 'Sports', 'Drama', 'School', 'Shoun..."
1,23273,Shigatsu wa Kimi no Uso,"[Music, accompanies, the, path, of, the, human...","['Drama', 'Music', 'Romance', 'School', 'Shoun..."
2,34599,Made in Abyss,"[The, Abyss—a, gaping, chasm, stretching, down...","['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F..."
3,5114,Fullmetal Alchemist: Brotherhood,"[""In, order, for, something, to, be, obtained,...","['Action', 'Military', 'Adventure', 'Comedy', ..."
4,31758,Kizumonogatari III: Reiketsu-hen,"[After, helping, revive, the, legendary, vampi...","['Action', 'Mystery', 'Supernatural', 'Vampire']"


In [134]:
anime.iloc[0].genre

"['Comedy', 'Sports', 'Drama', 'School', 'Shounen']"

In [135]:
type(anime.iloc[0].synopsis)

genre_list = anime['genre'].tolist() # contains all the genres of each anime 
syn_list = anime['synopsis'].tolist() # contains synopsis of each anime 

tags = [] # will be the concat of genre and synopsis of each anime 

for i in range(len(genre_list)):
    tags.append((" ".join(syn_list[i]) + genre_list[i]).lower()) # making the tag and lowercased it


In [136]:
# Clean the tags
cleaned_tags = []
for tag in tags: 
    tag = re.sub(r"[\[\]]",'',tag) # removing the square brackets 
    tag = remove_stopwords(tag) # removing stopwords
    
    cleaned_tags.append(tag)

In [137]:
cleaned_tags = pd.DataFrame(cleaned_tags)
anime["tags"] = cleaned_tags # appending a new col to the df

In [138]:
anime.head(5)

,uid,title,synopsis,genre,tags
0,28891,Haikyuu!! Second Season,"[Following, their, participation, at, the, Int...","['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","following participation inter-high, karasuno h..."
1,23273,Shigatsu wa Kimi no Uso,"[Music, accompanies, the, path, of, the, human...","['Drama', 'Music', 'Romance', 'School', 'Shoun...","music accompanies path human metronome, prodig..."
2,34599,Made in Abyss,"[The, Abyss—a, gaping, chasm, stretching, down...","['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","abyss—a gaping chasm stretching depths earth, ..."
3,5114,Fullmetal Alchemist: Brotherhood,"[""In, order, for, something, to, be, obtained,...","['Action', 'Military', 'Adventure', 'Comedy', ...","""in order obtained, equal value lost."" alchemy..."
4,31758,Kizumonogatari III: Reiketsu-hen,"[After, helping, revive, the, legendary, vampi...","['Action', 'Mystery', 'Supernatural', 'Vampire']",helping revive legendary vampire kiss-shot ace...


In [139]:
new = anime[['uid', 'title', 'tags']].copy() # created a new data frame w/ the cols we will work with 

In [140]:
new.head(5)

,uid,title,tags
0,28891,Haikyuu!! Second Season,"following participation inter-high, karasuno h..."
1,23273,Shigatsu wa Kimi no Uso,"music accompanies path human metronome, prodig..."
2,34599,Made in Abyss,"abyss—a gaping chasm stretching depths earth, ..."
3,5114,Fullmetal Alchemist: Brotherhood,"""in order obtained, equal value lost."" alchemy..."
4,31758,Kizumonogatari III: Reiketsu-hen,helping revive legendary vampire kiss-shot ace...


In [141]:
new['tags'][0]

"following participation inter-high, karasuno high school volleyball team attempts refocus efforts, aiming conquer spring tournament instead. receive invitation long-standing rival nekoma high, karasuno agrees large training camp alongside notable volleyball teams tokyo national level players. playing toughest teams japan, hope sharpen skills, come new attacks strengthen them. moreover, hinata kageyama attempt devise powerful weapon, possibly break sturdiest blocks. facing chance victory senior players graduate, members karasuno's volleyball team learn settle differences train harder hope overcome formidable opponents old new—including archrival aoba jousai world-class setter tooru oikawa. written mal rewrite'comedy', 'sports', 'drama', 'school', 'shounen'"

In [142]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [143]:
vectors = cv.fit_transform(new['tags'].values.astype('U')).toarray()

In [144]:
vectors 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [145]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [146]:
len(cv.get_feature_names())

/Users/tasnimmahi/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


5000